In [1]:
import numpy as np
from matplotlib import pyplot as plt
from numba import njit,extending,prange
from math import sin,cos,pi,atan2,atan,sqrt
from ipywidgets import interact

In [133]:
#smoothstep(10,11,np.array([2,3]))
x=np.array([2,3])
t=(x-10)/(11-10)
t=npclip(t,0.,1.)
t**2*(3-2*t)

array([0., 0.])

In [3]:
@njit
def fract(p): #returns fractional part
    return p-np.floor(p)
@njit
def clip(a, a_min, a_max):
    out=0.
    if a<a_min:
        out=a_min
    elif a>a_max:
        out=a_max
    else:
        out=a
    return out
@njit('double[:](double[:], double,double)')
def npclip(a, a_min, a_max):   
    out=np.zeros_like(a)
    for i in range(a.shape[0]):
        if a[i] < a_min:
            out[i] = a_min
        elif a[i] > a_max:
            out[i] = a_max
        else:
            out[i] = a[i]
    return out

@njit
#pseudo-random function, returns value between [0.,1.]
def hash(p):
    return fract(np.sin(833443.123456*\
        p*np.array([234.45,436.378])))
@njit
def smoothstep1D(edge0,edge1,x):
    assert edge0<edge1
    t=(x-edge0)/(edge1-edge0)
    t=clip(t,0.,1.)
    return t**2*(3.-2*t)
@njit
def smoothstep2D(edge0,edge1,x):
    assert edge0<edge1
    t=(x-edge0)/(edge1-edge0)
    t=npclip(t,0.,1.)
    return t**2*(3.-2*t)
# bilinear value noise function
@njit
def noise(st):
    i = np.floor(st)
    f = fract(st)
    # Four corners of a 2D square
    f00 = hash(i)
    f10 = hash(i + np.array([1.0, 0.0]))
    f01 = hash(i + np.array([0.0, 1.0]))
    f11 = hash(i + np.array([1.0, 1.0]))
    u = smoothstep2D(0.,1.,(1.-f))
    return u[0]*u[1]*f00+(1.-u[0])*u[1]*f10+\
     u[0]*(1.-u[1])*f01+(1.-u[0])*(1.-u[1])*f11

In [4]:
noise(np.array([.2,.5]))

array([0.00476284, 0.23081877])

In [29]:
width,height=640,480
aspect=width/height
image=np.zeros((height,width),dtype='int')
@njit
def circle(freq,image):
    uv=np.ones(2)
    for x in prange(width):
        for y in prange(height):
            uv[:2]=x/width,y/height
            uv=2*uv-1.;uv[0]*=aspect
            #uv*=noise(10.*uv)+.5*noise(5.*uv)
            d=sqrt(uv[0]**2+uv[1]**2)
            angle=atan2(uv[1],uv[0])
            uv=noise(20.*np.array([d,angle]))
            d=sqrt(uv[0]**2+uv[1]**2)

            #d+=.1*sin(15*angle)
            #image[y,x]=int(255*abs(sin(freq*d)))
            image[y,x]=int(255*abs(sin(freq*d)))

def run(freq):
    circle(freq,image)
    plt.imshow(image,cmap='gray')
    plt.axis('off');plt.show()    
interact(run,freq=(1.,20.,2.))

interactive(children=(FloatSlider(value=9.0, description='freq', max=20.0, min=1.0, step=2.0), Output()), _dom…

<function __main__.run(freq)>

In [58]:
image.min(),gray

(0, 0.0)